In [11]:
class Field:
    def __init__(self, name, column=None, primary_key=False, unique=False, index=False, nullable=True, default=None):
        self.name = name
        self.column = column
        self.primary_key = primary_key
        self.unique = unique
        self.index = index
        self.nullable = nullable
        self.default = default
        
    def validate(self, value) -> bool:
        raise NotImplemented
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        # n
        return instance.__dict__.get(self.name)
        # en
    
    def __set__(self, instance, value):
        # 这里是一个基类，所以我们并不知道他要干嘛，所以写一个空的
        self.validate(value)
        instance.__dict__[self.name] = value

In [17]:
class IntField(Field):
    def __init__(self, name, column=None, primary_key=False, unique=False, index=False, nullable=True, default=None, auto_increasement=False):
        super().__init__(name, column, primary_key, unique, index, nullable, default)
        self.auto_increasement = auto_increasement
        
    def validate(self, value):
#         if not self.nullable and value is None:
#             raise TypeError('{}<{}> required'.format(self.name, self.column))
        if value is None:
            return
        if not isinstance(value, int):
            raise TypeError('{}<{}> must be int but {}'.format(self.name, self.column, type(value)))

In [16]:
class StringField(Field):
    def __init__(self, name, column=None, primary_key=False, unique=False, index=False, nullable=True, default=None, length=45):
        super().__init__(name, column, primary_key, unique, index, nullable, default)
        self.length = length
        
    def validate(self, value):
        if value is None:
#             if self.nullable:
#                 return 
#             raise TypeError('{}<{}> required'.format(self.name, self.column))
            return
        if not isinstance(value, str):
            raise TypeError('{}<{}> must be str but {}'.format(self.name, self.column, type(value)))
        if len(value) > self.length:
            raise ValueError('{}<{}> too long'.format(self.name, self.column))

In [30]:
class Model:
    # 这里定义一个通用的save方法
    def save(self):
        fields = {}
        for name, field in self.__class__.__dict__.items():
            if isinstance(field, Field):
                fields[name] = field
        keys = []
        values = []
        # 接下来就要拼我们的sql了
        for name, value in self.__dict__.items():
            if name in fields.keys():
                keys.append('`{}`'.format(name))
                values.append(value)
        print(keys)
        print(values)
        # 这2个循环出来之后
        query = '''insert into `{}` ({}) values ({}) '''.format(self.__class__.__table__, ','.join(keys), ','.join(['%s'] * len(keys)))
        print(query)   

In [31]:
class User(Model):
    __table__ = 'user'
    
    id = IntField(name='id', column='id', primary_key=True, auto_increasement=True)
    name = StringField(name='name', column='name', nullable=False, unique=True, length=64)
    age = IntField(name='age', column='age')
    
    def __init__(self, id, name, age):
        self.id = id
        self.name = name
        self.age = age

In [32]:
user = User(10, 'miracle', 18)

In [33]:
user.save()

['`id`', '`name`', '`age`']
[10, 'miracle', 18]
insert into `user` (`id`,`name`,`age`) values (%s,%s,%s) 


In [7]:
class User:
    id = IntField(name='id', column='id', primary_key=True, auto_increasement=True)
    name = StringField(name='name', column='name', nullable=False, unique=True, length=64)
    age = IntField(name='age', column='age')
    
    def __init__(self, id, name, age):
        self.id = id
        self.name = name
        self.age = age
        
    def save(self, session):
        query = '''insert into `user` (`id`, `name`, `age`) values (%s, %s, %s)'''
        # n
        with session:
        # en
            session.execute(query, self.id, self.name, self.age)

In [8]:
class Session:
    def __init__(self, conn):
        self.conn = conn
        sel.cur = None
        
    def __enter__(self):
        self.cur = self.conn.cursor()
        return self
    
    def __exit__(self, *args):
        self.cur.close()
        self.conn.commit()
        
    def execute(query, *args):
        self.cur.execute(query, args)

In [34]:
class ModelMeta(type):
    def __new__(cls, name, bases, attrs):
        if '__table__' not in attrs.keys():
            attrs['__table__'] = name
        
        mapping = {}
        primary_key = []
        for k, v in attrs.items():
            if isinstance(v, Field):
                v.name = k
                if v.column is None:
                    v.column = k
                mapping[k] = v
                if v.primary_key:
                    primary_key.append(v)
                    
        attrs['__mapping__'] = mapping
        attrs['__primary_key__'] = primary_key
    
        return super().__new__(cls, name, bases, attrs)


class Model(metaclass=ModelMeta):
    pass

In [35]:
from sqlalchemy import create_engine

In [37]:
engine = create_engine('mysql+pymysql://root@127.0.0.1:3306/demo', echo=True)

In [38]:
from sqlalchemy.ext.declarative import declarative_base

In [40]:
Base = declarative_base()

In [42]:
from sqlalchemy import Column, Integer, String

In [61]:
class User(Base):
    # 昨天叫table，在这里叫tablename
    __tablename__ = 'user'

    # 这里需要引入一个Column，这里他把Column作为一个描述器，每种类型作为这里的参数
    # 第一个参数是类型，第二个是主键，我们可以help看一下help(Column)
    # 这里非常的长，我们一个个看，把我们用的比较多的都看下，看到__repr__
    # 这里还需要设置主键吧
    # 这里我们可以指定name是user_id,但是我个人习惯数据库的名字和python里是一样的
    id = Column(Integer, autoincrement=True, primary_key=True)
    name = Column(String(64), unique=True, nullable=False)
    age = Column(Integer)

    # 不是必须的这个方法，通常在生产上不写这个方法，写这个方法只是为了调试
    def __str__(self):
        return 'User(id={}, name={}, age={})'.format(self.id, self.name, self.age)

    # 具体的参数大家可以回去看

    # 这里的Model类不能重复定义的，如果之前有人定义过了，这里就需要一些选项了，或者就直接重新定义一个Base
    # 正常情况是不会的，因为我们不会在ipython里敲代码写项目吧

/Users/miracleYoung/.pyenv/versions/3.6.1/envs/miracle_3.6/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'user' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [57]:
Base.metadata.create_all(engine)

2017-11-21 22:07:21,787 INFO sqlalchemy.engine.base.Engine DESCRIBE `user`
2017-11-21 22:07:21,787 INFO sqlalchemy.engine.base.Engine {}
2017-11-21 22:07:21,790 INFO sqlalchemy.engine.base.Engine ROLLBACK
2017-11-21 22:07:21,792 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(64) NOT NULL, 
	age INTEGER, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2017-11-21 22:07:21,792 INFO sqlalchemy.engine.base.Engine {}
2017-11-21 22:07:21,808 INFO sqlalchemy.engine.base.Engine COMMIT


In [47]:
Base.metadata.drop_all(engine)

2017-11-21 22:04:34,147 INFO sqlalchemy.engine.base.Engine DESCRIBE `user`
2017-11-21 22:04:34,148 INFO sqlalchemy.engine.base.Engine {}
2017-11-21 22:04:34,152 INFO sqlalchemy.engine.base.Engine 
DROP TABLE user
2017-11-21 22:04:34,152 INFO sqlalchemy.engine.base.Engine {}
2017-11-21 22:04:34,155 INFO sqlalchemy.engine.base.Engine COMMIT


In [48]:
from sqlalchemy.orm import sessionmaker

In [59]:
session.close()

In [49]:
Session = sessionmaker(bind=engine)

In [60]:
session = Session()

In [62]:
user = User()

In [63]:
user.name = 'miracle'

In [64]:
user.age = 18

In [65]:
print(user)

User(id=None, name=miracle, age=18)


In [66]:
session.add(user)

In [67]:
session.commit()

2017-11-22 07:15:37,238 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:15:37,239 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:15:37,240 INFO sqlalchemy.engine.base.Engine {'name': 'miracle', 'age': 18}
2017-11-22 07:15:37,244 INFO sqlalchemy.engine.base.Engine COMMIT


In [68]:
print(user)

2017-11-22 07:15:49,511 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:15:49,513 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-22 07:15:49,514 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
User(id=1, name=miracle, age=18)


In [69]:
print(user)

User(id=1, name=miracle, age=18)


In [70]:
user._sa_instance_state.dict

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10a90b8d0>,
 'age': 18,
 'id': 1,
 'name': 'miracle'}

In [71]:
user2 = User()
user2.name = 'miracle'
user2.age = 10
session.add(user2)

In [72]:
session.commit()

2017-11-22 07:18:21,947 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:18:21,948 INFO sqlalchemy.engine.base.Engine {'name': 'miracle', 'age': 10}
2017-11-22 07:18:21,950 INFO sqlalchemy.engine.base.Engine ROLLBACK


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'miracle' for key 'name'") [SQL: 'INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)'] [parameters: {'name': 'miracle', 'age': 10}]

In [73]:
user3 = User()
user3.name = 'young'
user3.age = 5
session.add(user3)
session.commit()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'miracle' for key 'name'") [SQL: 'INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)'] [parameters: {'name': 'miracle', 'age': 10}]

In [74]:
session.rollback()

/Users/miracleYoung/.pyenv/versions/3.6.1/envs/miracle_3.6/lib/python3.6/site-packages/sqlalchemy/orm/session.py:507: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [75]:
session.add(user3)

In [76]:
session.commit()

2017-11-22 07:19:16,394 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:19:16,395 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:19:16,396 INFO sqlalchemy.engine.base.Engine {'name': 'young', 'age': 5}
2017-11-22 07:19:16,398 INFO sqlalchemy.engine.base.Engine COMMIT


In [77]:
print(user)

2017-11-22 07:19:54,216 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:19:54,218 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-22 07:19:54,218 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
User(id=1, name=miracle, age=18)


In [78]:
user.age = 20

In [79]:
session.add(user)

In [80]:
session.commit()

2017-11-22 07:20:05,857 INFO sqlalchemy.engine.base.Engine UPDATE user SET age=%(age)s WHERE user.id = %(user_id)s
2017-11-22 07:20:05,858 INFO sqlalchemy.engine.base.Engine {'age': 20, 'user_id': 1}
2017-11-22 07:20:05,860 INFO sqlalchemy.engine.base.Engine COMMIT


In [81]:
u = User()

In [82]:
u.id = 1
u.name = 'miracle'
u.age = 17

In [83]:
session.add(u)

In [84]:
session.commit()

2017-11-22 07:20:56,382 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:20:56,384 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-22 07:20:56,385 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2017-11-22 07:20:56,386 INFO sqlalchemy.engine.base.Engine ROLLBACK


FlushError: New instance <User at 0x10abedfd0> with identity key (<class '__main__.User'>, (1,)) conflicts with persistent instance <User at 0x10a90b4e0>

In [85]:
session.delete(user3)

In [86]:
session.commit()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: New instance <User at 0x10abedfd0> with identity key (<class '__main__.User'>, (1,)) conflicts with persistent instance <User at 0x10a90b4e0>

In [87]:
session.rollback()

/Users/miracleYoung/.pyenv/versions/3.6.1/envs/miracle_3.6/lib/python3.6/site-packages/sqlalchemy/orm/session.py:507: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [88]:
session.delete(user3)

In [89]:
session.commit()

2017-11-22 07:21:50,461 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:21:50,462 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-22 07:21:50,464 INFO sqlalchemy.engine.base.Engine {'param_1': 3}
2017-11-22 07:21:50,465 INFO sqlalchemy.engine.base.Engine DELETE FROM user WHERE user.id = %(id)s
2017-11-22 07:21:50,466 INFO sqlalchemy.engine.base.Engine {'id': 3}
2017-11-22 07:21:50,467 INFO sqlalchemy.engine.base.Engine COMMIT


In [90]:
session.delete(u)

InvalidRequestError: Instance '<User at 0x10abedfd0>' is not persisted

In [91]:
session.rollback()

In [92]:
u._sa_instance_state.deleted

False

In [93]:
user._sa_instance_state.deleted

False

In [94]:
user3._sa_instance_state.deleted

False

In [95]:
q = session.query(User)

In [96]:
for u in q:
    print(u)

2017-11-22 07:23:56,472 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user
2017-11-22 07:23:56,473 INFO sqlalchemy.engine.base.Engine {}
User(id=1, name=miracle, age=20)


In [98]:
import random

In [99]:
for x in range(100):
    user = User()
    user.name = 'user_{}'.format(x)
    user.age = random.randint(0, 100)
    session.add(user)
session.commit()

2017-11-22 07:24:21,085 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,086 INFO sqlalchemy.engine.base.Engine {'name': 'user_0', 'age': 98}
2017-11-22 07:24:21,087 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,088 INFO sqlalchemy.engine.base.Engine {'name': 'user_1', 'age': 2}
2017-11-22 07:24:21,089 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,089 INFO sqlalchemy.engine.base.Engine {'name': 'user_2', 'age': 61}
2017-11-22 07:24:21,090 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,091 INFO sqlalchemy.engine.base.Engine {'name': 'user_3', 'age': 38}
2017-11-22 07:24:21,092 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,093 INFO sqlalchemy.engine.base.Engine {'name': 'u

2017-11-22 07:24:21,148 INFO sqlalchemy.engine.base.Engine {'name': 'user_40', 'age': 18}
2017-11-22 07:24:21,148 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,149 INFO sqlalchemy.engine.base.Engine {'name': 'user_41', 'age': 86}
2017-11-22 07:24:21,150 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,150 INFO sqlalchemy.engine.base.Engine {'name': 'user_42', 'age': 16}
2017-11-22 07:24:21,151 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,152 INFO sqlalchemy.engine.base.Engine {'name': 'user_43', 'age': 96}
2017-11-22 07:24:21,153 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,153 INFO sqlalchemy.engine.base.Engine {'name': 'user_44', 'age': 54}
2017-11-22 07:24:21,154 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VA

2017-11-22 07:24:21,213 INFO sqlalchemy.engine.base.Engine {'name': 'user_80', 'age': 85}
2017-11-22 07:24:21,214 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,215 INFO sqlalchemy.engine.base.Engine {'name': 'user_81', 'age': 51}
2017-11-22 07:24:21,216 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,216 INFO sqlalchemy.engine.base.Engine {'name': 'user_82', 'age': 51}
2017-11-22 07:24:21,217 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,218 INFO sqlalchemy.engine.base.Engine {'name': 'user_83', 'age': 33}
2017-11-22 07:24:21,219 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-22 07:24:21,219 INFO sqlalchemy.engine.base.Engine {'name': 'user_84', 'age': 69}
2017-11-22 07:24:21,220 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VA

In [100]:
q = q.filter(User.age < 18) # 增加where子句
for u in q:
    print(u)

2017-11-22 07:25:12,349 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-22 07:25:12,351 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s
2017-11-22 07:25:12,351 INFO sqlalchemy.engine.base.Engine {'age_1': 18}
User(id=5, name=user_1, age=2)
User(id=10, name=user_6, age=2)
User(id=15, name=user_11, age=1)
User(id=22, name=user_18, age=16)
User(id=23, name=user_19, age=9)
User(id=32, name=user_28, age=1)
User(id=33, name=user_29, age=16)
User(id=39, name=user_35, age=0)
User(id=40, name=user_36, age=12)
User(id=46, name=user_42, age=16)
User(id=57, name=user_53, age=16)
User(id=58, name=user_54, age=0)
User(id=63, name=user_59, age=4)
User(id=64, name=user_60, age=8)
User(id=75, name=user_71, age=5)
User(id=89, name=user_85, age=12)
User(id=90, name=user_86, age=3)
User(id=97, name=user_93, age=14)


In [102]:
for u in session.query(User).filter(User.age < 18).filter(User.age > 12): # 我们可以多个filter，这个连续多个filter就像我们的and关系
    print(u)

2017-11-22 07:25:53,372 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s AND user.age > %(age_2)s
2017-11-22 07:25:53,373 INFO sqlalchemy.engine.base.Engine {'age_1': 18, 'age_2': 12}
User(id=22, name=user_18, age=16)
User(id=33, name=user_29, age=16)
User(id=46, name=user_42, age=16)
User(id=57, name=user_53, age=16)
User(id=97, name=user_93, age=14)


In [103]:
from sqlalchemy import or_

In [104]:
for u in session.query(User).filter(or_(User.age < 18, User.age > 70)): # 这里就是用了or_方法了，在filter里面，加了or之后，都放在里面，多个就用逗号分隔
    print(u)


2017-11-22 07:26:26,039 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s OR user.age > %(age_2)s
2017-11-22 07:26:26,040 INFO sqlalchemy.engine.base.Engine {'age_1': 18, 'age_2': 70}
User(id=4, name=user_0, age=98)
User(id=5, name=user_1, age=2)
User(id=10, name=user_6, age=2)
User(id=11, name=user_7, age=93)
User(id=12, name=user_8, age=84)
User(id=15, name=user_11, age=1)
User(id=17, name=user_13, age=90)
User(id=21, name=user_17, age=96)
User(id=22, name=user_18, age=16)
User(id=23, name=user_19, age=9)
User(id=24, name=user_20, age=98)
User(id=26, name=user_22, age=93)
User(id=27, name=user_23, age=75)
User(id=28, name=user_24, age=96)
User(id=30, name=user_26, age=85)
User(id=32, name=user_28, age=1)
User(id=33, name=user_29, age=16)
User(id=34, name=user_30, age=98)
User(id=36, name=user_32, age=92)
User(id=39, name=user_35, age=0)
User(id=40, name=user_36, age=12)
User(id=45, name=us